In [1]:
import json
import os
import math
import librosa
from tensorflow.keras.models import load_model
import numpy as np
from pydub import AudioSegment

import soundfile as sf
from pedalboard.io import AudioFile
from pedalboard import *
import noisereduce as nr
from tensorflow.image import resize
import sounddevice as sd
import time
import scipy.io.wavfile as wav
from pedalboard.io import AudioFile
from pedalboard import *



c:\Users\Ricky\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [32]:
print(sd.query_devices())

   0 Microsoft Sound Mapper - Input, MME (2 in, 0 out)
>  1 Microphone (WO Mic Device), MME (2 in, 0 out)
   2 Microphone Array (Realtek(R) Au, MME (2 in, 0 out)
   3 Headset (Terra 3 Mini), MME (2 in, 0 out)
   4 Microsoft Sound Mapper - Output, MME (0 in, 2 out)
<  5 Headphones (Terra 3 Mini), MME (0 in, 2 out)
   6 Speakers (Realtek(R) Audio), MME (0 in, 2 out)
   7 Primary Sound Capture Driver, Windows DirectSound (2 in, 0 out)
   8 Microphone (WO Mic Device), Windows DirectSound (2 in, 0 out)
   9 Microphone Array (Realtek(R) Audio), Windows DirectSound (2 in, 0 out)
  10 Headset (Terra 3 Mini), Windows DirectSound (2 in, 0 out)
  11 Primary Sound Driver, Windows DirectSound (0 in, 2 out)
  12 Headphones (Terra 3 Mini), Windows DirectSound (0 in, 2 out)
  13 Speakers (Realtek(R) Audio), Windows DirectSound (0 in, 2 out)
  14 Speakers (Realtek(R) Audio), Windows WASAPI (0 in, 2 out)
  15 Headphones (Terra 3 Mini), Windows WASAPI (0 in, 2 out)
  16 Microphone Array (Realtek(R) Audio

### Muscic Record 🎙️

In [219]:
sample_rate = 44100

def save_audio(filename, sample_rate, data):
    max_val = np.max(np.abs(data))
    if max_val > 0:
        data = data / max_val

    wav.write(filename, sample_rate, (data * 32767).astype(np.int16))


# print("Recording...")
# recording = np.zeros(0)

# def callback(indata, frames, time, status):
#     global recording
#     recording = np.append(recording, indata[:, 0])

# stream = sd.InputStream(
#     callback=callback, channels=1, samplerate=sample_rate, blocksize=1024
# )
# stream.start()

# start_time = time.time()
# while time.time() - start_time < 30:
#     time.sleep(0.5)  # Update every 100 ms
    
# stream.stop()
# stream.close()
# print("Recording finished.")

print("Mulai merekam...")
recording = sd.rec(int(30 * sample_rate), samplerate=sample_rate, channels=1, dtype='float64')
sd.wait()  # Tunggu hingga rekaman selesai
print("Rekaman selesai.")

save_audio("scipy.wav", sample_rate, recording)

with AudioFile('scipy.wav').resampled_to(sample_rate) as f:
    audio = f.read(f.frames)

# audio = nr.reduce_noise(y=audio, sr=sample_rate, stationary=True, prop_decrease=0.5)

board = Pedalboard([
    NoiseGate(threshold_db=-50, ratio=1.8, release_ms=250),
    Compressor(threshold_db=-15, ratio=2.5),
    # LowShelfFilter(cutoff_frequency_hz=400, gain_db=10, q=1),
    Gain(gain_db=10)
])

effected = board(audio, sample_rate)


with AudioFile('audio2_enhenced.wav', 'w', sample_rate, effected.shape[0]) as f:
    f.write(effected)

Mulai merekam...
Rekaman selesai.


### Model Init ⚙️

In [4]:
model_chitapatel = load_model("./model/custom_cnn_0.0031.h5")
model_tripathi = load_model("./model/tripathi_model.h5")
model_val_v1_2 = load_model("./model/cnn__genre_detection_0.93.h5")
model_val_v2_3 = load_model("./model/cnn__genre_detection_0.92.h5")
model_val_v2_4 = load_model("./model/cnn__genre_detection_44100hz_0.91.h5")
model_val_v2_6 = load_model("./model/cnn__genre_detection_41100hz_0.95(Tripathi Dataset).h5")


### Test Requirements 🧪

In [220]:
SAMPLE_RATE_1 = 22050
SAMPLE_RATE_2 = 44100

TRACK_DURATION = 30  # second
# file_path = "../mfcc_genre_detection/datasets/jazz/jazz.00037.wav"
# file_path = "../audio_tester/armada - hargai aku-[AudioTrimmer.com].mp3"
# file_path = "../../../Musics/2022+/SZA - Kill Bill (Audio).mp3"
# file_path = "../../../Musics/gama - satu ataw dua.mp3"
# file_path = "../../../Musics/2020s/Ava Max - Salt (Lyrics).mp3"
# file_path = "../../../Musics/2020s/Qorygore - Sampurasun (Lyrics) ft. Pretty Rico.mp3"
# file_path = "../../../Musics/2020s/Dj Quads - Punch.mp3"
# file_path = "../../../Musics/2020s/mxmtoon - ok on your own (ft. Carly Rae Jepsen)  audio .mp3"
file_path = "scipy.wav"
file_path_2 = "scipy_jazz.wav"
file_path_3 = "audio2_enhenced.wav"
# file_path_4 = "audio4_enhenced.wav"

genres = {
    "Blues": 0,
    "Classical": 0,
    "Country": 0,
    "Disco": 0,
    "HipHop": 0,
    "Jazz": 0,
    "Metal": 0,
    "Pop": 0,
    "Reggae": 0,
    "Rock": 0,
}

genres_list = list(genres.keys())

### Utils 🪄

In [6]:

# CHITAPATEL MDOEL UTILS
def splitsongs(X,window=0.05, overlap=0.5):
    # Empty lists to hold our results
    temp_X = []

    # Get the input song array size
    xshape = X.shape[0]
    chunk = int(xshape * window)
    offset = int(chunk * (1.0 - overlap))

    # Split the song and create new ones on windows
    spsong = [X[i : i + chunk] for i in range(0, xshape - chunk + offset, offset)]
    for s in spsong:
        if s.shape[0] != chunk:
            continue

        temp_X.append(s)

    return np.array(temp_X)

def to_melspectrogram(songs, n_fft=1024, hop_length=256):
    # Transformation function
    melspec = lambda x: librosa.feature.melspectrogram(
        y=x, sr=22050, n_fft=n_fft, hop_length=hop_length, n_mels=128
    )[
        :, :, np.newaxis
    ]  # keep n_mels=128. other values are for experimenting

    # map transformation of input songs to melspectrogram using log-scale
    tsongs = map(melspec, songs)
    # np.array([librosa.power_to_db(s, ref=np.max) for s in list(tsongs)])
    return np.array(list(tsongs))


def chitapatel_service(model, file_path):
  sample_rate = 22000
  signal, sr = librosa.load(file_path, sr=sample_rate)

  # start_signal = (sample_rate * 60)
  # end_signal = (sample_rate * 90)
  # signal = signal[start_signal:end_signal]
  signal = signal[:sample_rate * 30]

  # signal = nr.reduce_noise(y=signal, sr=sample_rate)
  print(signal.shape)

  signals = splitsongs(signal, window=0.05)
  x = to_melspectrogram(signals)
  # x = np.array(x[0])
  # x = x[..., np.newaxis]
  # x = x[np.newaxis,...]
  print(x.shape)
  # x = x.reshape(1, 128, 129, 1)
  # print(x)
  prediction = model.predict(x)

  return prediction


# VALERIO MODEL UTILS
def extract_mfcc(file_path, sr,num_mfcc=13, n_fft=2048, hop_length=512, num_segments=5):
    data = []
    SAMPLES_PER_TRACK = sr  * TRACK_DURATION
    samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
    num_mfcc_vectors_per_segment = math.ceil(samples_per_segment / hop_length)

    signal, sample_rate = librosa.load(file_path, sr=sr)
    # signal = signal[(SAMPLE_RATE * 40):(SAMPLE_RATE * 70)]
    # signal = nr.reduce_noise(y=signal, sr=SAMPLE_RATE)
    for d in range(num_segments):
        # calculate start and finish sample for current segment
        start = samples_per_segment * d
        finish = start + samples_per_segment
        # extract mfcc
        mfcc = librosa.feature.mfcc(
            y=signal[start:finish],
            sr=sample_rate,
            n_mfcc=num_mfcc,
            n_fft=n_fft,
            hop_length=hop_length,
        )

        mfcc = mfcc.T
        # store only mfcc feature with expected number of vectors
        if len(mfcc) == num_mfcc_vectors_per_segment:
            data.append(mfcc.tolist())

    return data

def valerio_service(model,file_path, sr):
    val = extract_mfcc(file_path=file_path, num_segments=10, sr=sr)
    val = np.array(val)
    print(val.shape)
    val = val[..., np.newaxis]
    return model.predict(val)


# TRIPATHI MODEL UTILS
def load_and_preprocess_data(file_path, target_shape=(150, 150)):
    data = []
    audio_data, sample_rate = librosa.load(file_path, sr=None)
    # Perform preprocessing (e.g., convert to Mel spectrogram and resize)
    # Define the duration of each chunk and overlap
    chunk_duration = 4  # seconds
    overlap_duration = 2  # seconds
    print(sample_rate)
                
    # Convert durations to samples
    chunk_samples = chunk_duration * sample_rate
    overlap_samples = overlap_duration * sample_rate
                
    # Calculate the number of chunks
    num_chunks = int(np.ceil((len(audio_data) - chunk_samples) / (chunk_samples - overlap_samples))) + 1
                
    # Iterate over each chunk
    for i in range(num_chunks):
                    # Calculate start and end indices of the chunk
        start = i * (chunk_samples - overlap_samples)
        end = start + chunk_samples
                    
                    # Extract the chunk of audio
        chunk = audio_data[start:end]
                    
                    # Compute the Mel spectrogram for the chunk
        mel_spectrogram = librosa.feature.melspectrogram(y=chunk, sr=sample_rate)
                    
                #mel_spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate)
        mel_spectrogram = resize(np.expand_dims(mel_spectrogram, axis=-1), target_shape)
        data.append(mel_spectrogram)
    
    return np.array(data)


def tripathi_service(model, file_path):
    X_test = load_and_preprocess_data(file_path)
    # print(X_test.shape)
    return  model.predict(X_test)

### Model Compile 🚀

In [221]:
pred_1 = chitapatel_service(model_chitapatel, file_path)
pred_2 = tripathi_service(model_tripathi, file_path)
pred_6 = valerio_service(model_val_v1_2, file_path, SAMPLE_RATE_1)
pred_8 = valerio_service(model_val_v2_3, file_path, SAMPLE_RATE_2)
pred_9 = valerio_service(model_val_v2_4, file_path, SAMPLE_RATE_2)
pred_11 =tripathi_service(model_val_v2_6, file_path) 

# pred_1 = chitapatel_service(model_chitapatel, file_path_2)
# pred_2 = tripathi_service(model_tripathi, file_path_2)
# pred_6 = valerio_service(model_val_v1_2, file_path_2, SAMPLE_RATE_1)
# pred_8 = valerio_service(model_val_v2_3, file_path_2, SAMPLE_RATE_2)
# pred_9 = valerio_service(model_val_v2_4, file_path_2, SAMPLE_RATE_2)
# pred_11 =tripathi_service(model_val_v2_6, file_path_2) 

(660000,)
(39, 128, 129, 1)
2/2 [==============================] - 0s 24ms/step
44100
1/1 [==============================] - 0s 366ms/step
(10, 130, 13)
1/1 [==============================] - 0s 56ms/step
(10, 259, 13)
1/1 [==============================] - 0s 56ms/step
(10, 259, 13)
1/1 [==============================] - 0s 53ms/step
44100
1/1 [==============================] - 0s 86ms/step


### Evaluation 📐

In [223]:
def model_evaluation(prediction):
  # print(prediction.shape[0])
  for i in range(prediction.shape[0]):
    #     genres[current_genre] = value

    for key, v in enumerate(prediction[i]):

        # genres[genres_list[key]] = genres[genres_list[key]] + v
        genres[genres_list[key]] = ((genres[genres_list[key]] * 3)+ v) / 4

  print("\n\n<<========= Genre Detection Accuracy =======>>")
  # print(genres)
  total_val = 0
  for genre in genres:
      print("{}\t\t==> {}%".format(genre, round(genres[genre] * 100, 2)))
      total_val = total_val + genres[genre]

  print("total_val ==>", total_val)

def model_evaluation_valerio(prediction):
  # print(prediction.shape[0])
  for i in range(prediction.shape[0]):
    #     genres[current_genre] = value

    for key, v in enumerate(prediction[i]):

        # genres[genres_list[key]] = genres[genres_list[key]] + v
        genres[genres_list[key]] = ((genres[genres_list[key]] * 3)+ v) / 4

  print("\n\n<<========= Genre Detection Accuracy =======>>")
  # print(genres)
  total_val = 0
  for genre in genres:
      print("{}\t\t==> {}%".format(genre, round(genres[genre] * 100, 2)))
      total_val = total_val + genres[genre]

  print("total_val ==>", total_val)


print('\n'+'=======//======='*3)
print('\t\tCHITAPATEL MODEL')
print('=======//======='*3)
model_evaluation(pred_1)

print('\n\n'+'=======//======='*3)
print('\t\tTRIPATHI MODEL')
print('=======//======='*3)
model_evaluation(pred_2)

print('\n'+'=======//======='*3)
print('\t\tVALERIO MODEL (22050Hz)')
print('=======//======='*3)
model_evaluation_valerio(pred_6)

print('\n'+'=======//======='*3)
print('\t\tVALERIO MODEL (44100Hz)')
print('=======//======='*3)
model_evaluation_valerio(pred_8)
model_evaluation_valerio(pred_9)
model_evaluation_valerio(pred_11)



=======//==============//==============//=======
		CHITAPATEL MODEL
=======//==============//==============//=======


<<========= Genre Detection Accuracy =======>>
Blues		==> 20.0%
Classical		==> 0.02%
Country		==> 8.73%
Disco		==> 9.57%
HipHop		==> 6.22%
Jazz		==> 0.09%
Metal		==> 18.73%
Pop		==> 1.36%
Reggae		==> 7.86%
Rock		==> 27.43%
total_val ==> 1.0000000139658027


=======//==============//==============//=======
		TRIPATHI MODEL
=======//==============//==============//=======


<<========= Genre Detection Accuracy =======>>
Blues		==> 53.32%
Classical		==> 0.01%
Country		==> 13.69%
Disco		==> 5.68%
HipHop		==> 19.62%
Jazz		==> 0.01%
Metal		==> 5.34%
Pop		==> 0.29%
Reggae		==> 0.25%
Rock		==> 1.79%
total_val ==> 1.0000000097090715

=======//==============//==============//=======
		VALERIO MODEL (22050Hz)
=======//==============//==============//=======


<<========= Genre Detection Accuracy =======>>
Blues		==> 21.94%
Classical		==> 0.5%
Country		==> 0.84%
Disco		==> 0.39%


### ⏬